# Part A

### 1. Perform Exploratory Data Analysis (EDA) and discuss the data and what you observe prior to beginning modeling and how impact how to proceed [10 pts]

In [966]:
# import required packages

import pandas as pd
import numpy as np

import nltk
from time import time
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import category_encoders as ce
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC

from sklearn import svm

from sklearn.linear_model import ElasticNet

from sklearn.linear_model import SGDClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

import xgboost as xgb
import multiprocessing

In [967]:
# load the data

df = pd.read_csv('8k_diabetes.csv')


In [968]:
# preview the data
df.head(10).T

0  \
race                                                Caucasian   
gender                                                 Female   
age                                                   [50-60)   
weight                                                      ?   
admission_type_id                                    Elective   
discharge_disposition_id                   Discharged to home   
admission_source_id                        Physician Referral   
time_in_hospital                                            1   
payer_code                                                 CP   
medical_specialty                               Surgery-Neuro   
num_lab_procedures                                         35   
num_procedures                                              4   
num_medications                                            21   
number_outpatient                                           0   
number_emergency                                            0   
number_inpatient                                            0   
diag_1                                                    723   
diag_2                                                    723   
diag_3                                                    719   
number_diagnoses                                            9   
max_glu_serum                                            None   
A1Cresult                                                None   
metformin                                                  No   
repaglinide                                                No   
nateglinide                                                No   
chlorpropamide                                             No   
glimepiride                                                No   
acetohexamide                                              No   
glipizide                                                  No   
glyburide                                                  No   
tolbutamide                                                No   
pioglitazone                                               No   
rosiglitazone                                              No   
acarbose                                                   No   
miglitol                                                   No   
troglitazone                                               No   
tolazamide                                                 No   
examide                                                    No   
citoglipton                                                No   
insulin                                                    No   
glyburide.metformin                                        No   
glipizide.metformin                                        No   
glimepiride.pioglitazone                                   No   
metformin.rosiglitazone                                    No   
metformin.pioglitazone                                     No   
change                                                     No   
diabetesMed                                                No   
readmitted                                              False   
diag_1_desc                Spinal stenosis in cervical region   
diag_2_desc                Spinal stenosis in cervical region   
diag_3_desc               Effusion of joint, site unspecified   

                                                                          1  \
race                                                              Caucasian   
gender                                                               Female   
age                                                                 [20-30)   
weight                                                              [50-75)   
admission_type_id                                                    Urgent   
discharge_disposition_id                                 Discharged to home   
admission_source_id                                      Physician Referral   
time_in_hospital                               

In [969]:
# Checking the counts and Data types of each column 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   race                      8000 non-null   object
 1   gender                    8000 non-null   object
 2   age                       8000 non-null   object
 3   weight                    8000 non-null   object
 4   admission_type_id         7424 non-null   object
 5   discharge_disposition_id  7627 non-null   object
 6   admission_source_id       7250 non-null   object
 7   time_in_hospital          8000 non-null   int64 
 8   payer_code                8000 non-null   object
 9   medical_specialty         8000 non-null   object
 10  num_lab_procedures        8000 non-null   int64 
 11  num_procedures            8000 non-null   int64 
 12  num_medications           8000 non-null   int64 
 13  number_outpatient         8000 non-null   int64 
 14  number_emergency        

In [970]:
# Checking null values in all the columns 
# Here I want to see the number of nulls.
df.isnull().sum()

race                          0
gender                        0
age                           0
weight                        0
admission_type_id           576
discharge_disposition_id    373
admission_source_id         750
time_in_hospital              0
payer_code                    0
medical_specialty             0
num_lab_procedures            0
num_procedures                0
num_medications               0
number_outpatient             0
number_emergency              0
number_inpatient              0
diag_1                        0
diag_2                        0
diag_3                        0
number_diagnoses              0
max_glu_serum                 0
A1Cresult                     0
metformin                     0
repaglinide                   0
nateglinide                   0
chlorpropamide                0
glimepiride                   0
acetohexamide                 0
glipizide                     0
glyburide                     0
tolbutamide                   0
pioglita

On the previous step, I discovered that these columns contain null values.

admission_type_id
discharge_disposition_id 
admission_source_id
diag_2_desc
diag_3_desc

The null values from these columns can be replaced with '?', which is the simbol for 'Missing Values' in the data set.

For the missing values of diag_1_desc, diag_2_desc and diag_1_desc, I will replace the blanks with "".

In [971]:
# This loop will print the counts of each class in every column

print("All value counts by column")
for column in df.columns:
    print(column,"\n")
    
    print(df[column].value_counts(),"\n")
    

All value counts by column
race 

Caucasian          5891
AfricanAmerican    1639
?                   180
Hispanic            146
Other               101
Asian                43
Name: race, dtype: int64 

gender 

Female    4314
Male      3686
Name: gender, dtype: int64 

age 

[70-80)     2049
[60-70)     1749
[50-60)     1391
[80-90)     1248
[40-50)      800
[30-40)      340
[90-100)     214
[20-30)      117
[10-20)       65
[0-10)        27
Name: age, dtype: int64 

weight 

?            7675
[75-100)      138
[50-75)        95
[100-125)      61
[125-150)      12
[25-50)        10
[0-25)          5
[150-175)       4
Name: weight, dtype: int64 

admission_type_id 

Emergency        3968
Urgent           1545
Elective         1401
Not Available     476
Not Mapped         33
Newborn             1
Name: admission_type_id, dtype: int64 

discharge_disposition_id 

Discharged to home                                                                                             4879
Discharg

In [972]:
# This loop will print the unique classes of each column 

print("All classes by column")
for column in df.columns:
    print(column)
    print(df[column].drop_duplicates())

All classes by column
race
0            Caucasian
3      AfricanAmerican
13            Hispanic
49               Other
91               Asian
132                  ?
Name: race, dtype: object
gender
0    Female
2      Male
Name: gender, dtype: object
age
0       [50-60)
1       [20-30)
2       [80-90)
5       [70-80)
6       [60-70)
16      [30-40)
20      [40-50)
25      [10-20)
29     [90-100)
276      [0-10)
Name: age, dtype: object
weight
0               ?
1         [50-75)
54       [75-100)
62      [100-125)
233     [150-175)
260        [0-25)
660     [125-150)
1279      [25-50)
Name: weight, dtype: object
admission_type_id
0            Elective
1              Urgent
2       Not Available
3           Emergency
8                 NaN
242        Not Mapped
6953          Newborn
Name: admission_type_id, dtype: object
discharge_disposition_id
0                                      Discharged to home
2       Discharged/transferred to home with home healt...
6                             

The age column can be considered categorical data since the values represent a range of age. Maybe the values could be replaced with an age in the middle, for example 50-60 = 55.

In the dataset '?' represents mission values.
This code will show the count of '?' in all the columns.

In [973]:
for col in df.columns:
    if df[col].dtype == object:
         print(col,df[col][df[col] == '?'].count())

race 180
gender 0
age 0
weight 7675
admission_type_id 0
discharge_disposition_id 0
admission_source_id 0
payer_code 4260
medical_specialty 3229
diag_1 0
diag_2 55
diag_3 175
max_glu_serum 0
A1Cresult 0
metformin 0
repaglinide 0
nateglinide 0
chlorpropamide 0
glimepiride 0
acetohexamide 0
glipizide 0
glyburide 0
tolbutamide 0
pioglitazone 0
rosiglitazone 0
acarbose 0
miglitol 0
troglitazone 0
tolazamide 0
examide 0
citoglipton 0
insulin 0
glyburide.metformin 0
glipizide.metformin 0
glimepiride.pioglitazone 0
metformin.rosiglitazone 0
metformin.pioglitazone 0
change 0
diabetesMed 0
diag_1_desc 0
diag_2_desc 0
diag_3_desc 0


### 2. Pre-processed categorical data for use in the model and justified pre-processing method. Note this may be different for each algorithm you try. [10 pts]


For this task I decided to create functions that will allow to preprocess the dataset for Part A and Part B.

dropAndReplaceData -> Drops unused columns and replaces missing values '?' or blanks 

cleanText -> removes tokens like white spaces in the text, it also removes stops words from the text

getCatNumPipeline-> set ups a pipeline to transform the categorical data using CatBoostEncoder



In [974]:
def dropAndReplaceData(df):

    # Drop weight, payer_code, medical_specialty because they have a large number of missing values

    df = df.drop(['weight','payer_code','medical_specialty'], axis = 1)

    
    #In the following columns, there are 2 or 3 classes, however one of them has the majority of values

    #Tolbutamide, glipizide.metformin 
    #- The great majority of the data is NO, 7998. Only 2 are Steady
    df = df.drop(['tolbutamide', 'glipizide.metformin'], axis = 1)
    #Tolazamide
    #- The great majority of the data is NO, 7999. Only 1 is Steady
    df = df.drop(['tolazamide'], axis = 1)
    #Miglitol
    #- The great majority is NO 7997, Steady 2 and Down 1
    df = df.drop(['miglitol'], axis = 1)

    #Acarbose
    #- The majority is NO 7976, Steady 23 and Up 1
    df = df.drop(['acarbose'], axis = 1)

    #Chlorpropamide
    #- No 7990, Steady 9, Up 1
    df = df.drop(['chlorpropamide'], axis = 1)

    #Nateglinide
    #- No 7962, Steady 36, Down 1, Up 1
    df = df.drop(['nateglinide'], axis = 1)

    #Repaglinide
    #- No 7888, Steady 96, Up 11, Down 5
    df = df.drop(['repaglinide'], axis = 1)
    
    #acetohexamide,examide,citoglipton,Troglitazone, glimepiride.pioglitazone, metformin.rosiglitazone ,metformin.pioglitazone 

    df = df.drop(['acetohexamide','examide','citoglipton','troglitazone', 'glyburide.metformin' ,'glimepiride.pioglitazone', 'metformin.rosiglitazone' ,'metformin.pioglitazone'], axis = 1)
    
    
 

    #df = df.replace('?', 'Not Mapped')
    df = df.replace('?', 'Missing')


    #df['admission_type_id'].isnull().sum()

    # I decided to replace the null values in the following columns because the data on these rows could be important

    df['admission_type_id'].fillna("Not Mapped", inplace = True)
    df['discharge_disposition_id'].fillna("Not Mapped", inplace = True)
    df['admission_source_id'].fillna("Not Mapped", inplace = True)

    # In the case of the diagnostic description, I prefer to leave it as "" but not null. 
    df['diag_1_desc'].fillna("", inplace = True)
    df['diag_2_desc'].fillna("", inplace = True)
    df['diag_3_desc'].fillna("", inplace = True)


    df['age'] = df['age'].replace({"[70-80)":75,
                             "[60-70)":65,
                             "[50-60)":55,
                             "[80-90)":85,
                             "[40-50)":45,
                             "[30-40)":35,
                             "[90-100)":95,
                             "[20-30)":25,
                             "[10-20)":15,
                             "[0-10)":5})
    return df


In [975]:
df = dropAndReplaceData(df)

In [976]:
df

race  gender  age admission_type_id  \
0           Caucasian  Female   55          Elective   
1           Caucasian  Female   25            Urgent   
2           Caucasian    Male   85     Not Available   
3     AfricanAmerican  Female   55         Emergency   
4     AfricanAmerican  Female   55         Emergency   
...               ...     ...  ...               ...   
7995        Caucasian    Male   75          Elective   
7996        Caucasian  Female   55            Urgent   
7997        Caucasian    Male   55         Emergency   
7998        Caucasian    Male   85            Urgent   
7999        Caucasian  Female   75          Elective   

                               discharge_disposition_id  \
0                                    Discharged to home   
1                                    Discharged to home   
2     Discharged/transferred to home with home healt...   
3                                    Discharged to home   
4                                    Discharged to home   
...                                                 ...   
7995                      Discharged/transferred to SNF   
7996                                 Discharged to home   
7997  Discharged/transferred to another short term h...   
7998                      Discharged/transferred to SNF   
7999  Discharged/transferred to home with home healt...   

                             admission_source_id  time_in_hospital  \
0                             Physician Referral                 1   
1                             Physician Referral                 2   
2                                     Not Mapped                 7   
3     Transfer from another health care facility                 4   
4                                 Emergency Room                 5   
...                                          ...               ...   
7995                          Physician Referral                 8   
7996                              Emergency Room                 2   
7997                                  Not Mapped                 1   
7998                              Emergency Room                 7   
7999                    Transfer from a hospital                 6   

      num_lab_procedures  num_procedures  num_medications  ...  glyburide  \
0                     35               4               21  ...         No   
1                      8               5                5  ...         No   
2                     12               0               21  ...         No   
3                     33               1                5  ...         No   
4                     31               0               13  ...         No   
...                  ...             ...              ...  ...        ...   
7995                  56               2               27  ...         No   
7996                  44               0               18  ...         No   
7997                  25               0               16  ...         No   
7998                  61               0               22  ...         No   
7999                  48               0               15  ...         No   

      pioglitazone  rosiglitazone insulin change diabetesMed  readmitted  \
0               No             No      No     No          No       False   
1               No             No      No     No          No       False   
2               No             No  Steady     No         Yes        True   
3               No             No  Steady     No         Yes       False   
4               No             No  Steady     Ch         Yes       False   
...            ...            ...     ...    ...         ...         ...   
7995            No             No  Steady     No         Yes       False   
7996            No         Steady  Steady     Ch         Yes       False   
7997        Steady             No  Steady     Ch         Yes       False   
7998            No             No      Up     Ch         Yes       False   
7999            No             No

In [977]:
def cleanText(text):
    
    stop_words = set(stopwords.words("english")) 
    lemmatizer = WordNetLemmatizer()
    
    text = re.sub(r'[^A-Za-z0-9]+',' ',text)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    
    return text

def lematization(df):

    df['diag_1_desc'] = df.diag_1_desc.apply(lambda x: cleanText(x))

    df['diag_2_desc'] = df.diag_2_desc.apply(lambda x: cleanText(x))

    df['diag_3_desc'] = df.diag_3_desc.apply(lambda x: cleanText(x))
    
    return df



In [978]:
df = lematization(df)


In [979]:
# define the data preparation for the columns

def getCatNumPipeline(categorical_columns, numerical_columns):

    cat_encoder = ce.CatBoostEncoder()

    tfidf_params = dict(sublinear_tf= True, 
                           #min_df = 5, 
                           norm= 'l2', 
                           #ngram_range= (1,2), 
                           stop_words ='english')

    transformer = [
        ('cat_encoder', cat_encoder, categorical_columns), 
        #('num_scaler', MinMaxScaler(), numerical_columns),
        ('num_scaler', StandardScaler(), numerical_columns),

        #('tf_idf',TfidfVectorizer(**tfidf_params), text_columns)
        ]

    col_transform = ColumnTransformer(transformers=transformer)

    #text_model = LogisticRegression(solver="saga", penalty="elasticnet", l1_ratio=0.5)

    pipeline = Pipeline(steps=[('prep',col_transform)])
    
    return pipeline


### 3. Pre-processed numerical data appropriately including handling missing data and justified methods used. Note this may be different for each algorithm you try. [10 pts]

In [980]:
df.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns

Index(['age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses'],
      dtype='object')

The functions I created using pipeline encode the categorical data and apply Standard Scaler for numerical columns.
I considered that the numerical columns listed above were important in the dataset, so no numerical columns were dropped.

In [981]:
# split into inputs and outputs

X = df.drop(['readmitted'],axis =1)
y = df['readmitted'].astype(int)

# determine categorical and numerical features

numerical_columns = X.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns

categorical_columns = df.drop(['time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses','diag_1_desc','diag_2_desc','diag_3_desc','readmitted'],axis=1).columns
#X.select_dtypes(include=['object', 'bool']).columns

text_columns = ['diag_1_desc','diag_2_desc','diag_3_desc']


pipeline = getCatNumPipeline(categorical_columns, numerical_columns)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


pipeline.fit(X_train, y_train)

X_train_cat_num = pipeline.transform(X_train)

X_test_cat_num = pipeline.transform(X_test)

### 4. Implement a model to make predictions using text data using tf-idf [20 pts]


For this task I created a pipeline to transform the text columns using TF-IDF, and create a metamodel using Logistic Regression with Elasticnet.

In [982]:

def getTextPipeline():

    # get the params
    tfidf_params = dict(sublinear_tf= True, norm= 'l2', stop_words ='english')


    # create a Pipeline to transform features into numbers with TF-IDF, then pass to the model
    # and generate a prediction with LogisticRegression and Elasticnet penalty

    cls = Pipeline(steps=[
    ('features', TfidfVectorizer(**tfidf_params)),
    #('elasticnet', ElasticNet(random_state=0))
    ('clf', LogisticRegression(solver="saga", penalty="elasticnet", l1_ratio=0.5))
    ])
    
    return cls


In [983]:
def getTextMetamodel(X_train,y_train,X_test):
    
    # Use clf as a model, fit X_train and y_train
    cls.fit(X_train['diag_1_desc'], y_train)

    # predicted 
    text_pred_model1 = cls.predict_proba(X_test['diag_1_desc'])[:,1]
    #text_pred_model1 = cls.predict(X_test['Processed_diag_1_desc'])

    # Use clf as a model, fit X_train and y_train
    cls.fit(X_train['diag_2_desc'], y_train)

    # predicted 
    text_pred_model2 = cls.predict_proba(X_test['diag_2_desc'])[:,1]
    #text_pred_model2 = cls.predict(X_test['Processed_diag_2_desc'])

    # Use clf as a model, fit X_train and y_train
    cls.fit(X_train['diag_3_desc'], y_train)

    # predicted 
    text_pred_model3 = cls.predict_proba(X_test['diag_3_desc'])[:,1]
    #text_pred_model3 = cls.predict(X_test['Processed_diag_3_desc'])
    
    return text_pred_model1,text_pred_model2,text_pred_model3


In [984]:
cls = getTextPipeline()

### 5. Use model stacking to incorporate tf-idf predictions for all 3 text fields (so 3 models unless you elect to concatenate the text fields into 1 - need to justify if so) in downstream algorithm the uses non-text features [20 pts]

For this task I created the joinData function to merge the categorical and numerical data that was transformed earlier with the metamodel predictions. Notice that the predictions use probabilities, not True and False.

In [985]:
def joinData(cat_num,text_pred1,text_pred2,text_pred3):
    
    #numerical_columns and text_columns are "Global Variables". There is no need to use them as attributes
    
    columns = np.append(categorical_columns, numerical_columns)

    #df_cat_num = pd.DataFrame(data=X_test_cat_num,columns = columns)

    df_cat_num = pd.DataFrame(data=cat_num,columns = columns)

    #df_text_pred = pd.DataFrame(np.array([text_pred_model1,text_pred_model2,text_pred_model3]).T,columns = text_columns)
    df_text_pred = pd.DataFrame(np.array([text_pred_model1,text_pred_model2,text_pred_model3]).T,columns = text_columns)

    X = df_cat_num.join(df_text_pred)
    
    # pd.merge(df1, df2, left_index=True, right_index=True)
    
    #X = df_cat_num.join(df_text_pred)
    
    X = pd.merge(df_cat_num, df_text_pred, left_index=True, right_index=True)
    
    return X

In [986]:
text_pred_model1,text_pred_model2,text_pred_model3 = getTextMetamodel(X_train,y_train,X_test)

X = joinData(X_test_cat_num,text_pred_model1,text_pred_model2,text_pred_model3)

y = y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size=0.2, shuffle=False)


In [987]:
text_pred_model1

array([0.18938313, 0.29089583, 0.15887783, ..., 0.36469573, 0.52410871,
       0.45548817])

### 6. Perform experimentation for multiple modeling algorithms and justify why you selected the experiments you chose [20 pts]

Here I will test a couple of Algorithms to learn to learn what combination returns the best AUC score.

In [988]:
clf = RandomForestClassifier(n_estimators=150, random_state = 1)


clf.fit(X_train, y_train)

clf.score(X_test, y_test)

0.61875

In [989]:
pred_prob = clf.predict_proba(X_test)

auc_score1 = roc_auc_score(y_test, pred_prob[:,1])

auc_score1

0.6159307566511109

In [1012]:
clf = GradientBoostingClassifier(n_estimators=160, learning_rate=0.01,max_depth=40, subsample=0.6, random_state = 1, tol=0.1)

#clf = RandomForestClassifier(n_estimators=150)


clf.fit(X_train, y_train)

clf.score(X_test, y_test)

0.63125

In [1010]:
pred_prob = clf.predict_proba(X_test)

auc_score1 = roc_auc_score(y_test, pred_prob[:,1])

auc_score1

0.6212356906385864

In [992]:
clfGBC = GradientBoostingClassifier(n_estimators=160, learning_rate=0.01,max_depth=40, subsample=0.6, random_state = 1, tol=0.1)
    
scores = cross_val_score(clfGBC, X, y, cv=10,scoring='roc_auc')
scores

array([0.59919028, 0.55352227, 0.66396761, 0.68906883, 0.5697166 ,
       0.73813765, 0.64016194, 0.59684245, 0.62841797, 0.59570312])

In [993]:
clfGBC = GradientBoostingClassifier(n_estimators=150, 
                                    learning_rate=0.05,
                                    max_depth=20, 
                                    max_features='sqrt', 
                                    subsample=0.6, 
                                    validation_fraction=0.2,
                                    n_iter_no_change=5,
                                    tol=0.01,
                                    random_state = 1
                                   )
    
scores = cross_val_score(clfGBC, X, y, cv=10,scoring='roc_auc')
scores

array([0.61182186, 0.49311741, 0.55919028, 0.60323887, 0.57732794,
       0.65797571, 0.58898785, 0.57828776, 0.62695312, 0.5633138 ])

In [994]:
clfRFC = RandomForestClassifier(n_estimators=150, random_state = 1)
    
scores = cross_val_score(clfRFC, X, y, cv=10,scoring='roc_auc')
scores

array([0.63230769, 0.60623482, 0.66453441, 0.70242915, 0.55919028,
       0.73473684, 0.6534413 , 0.6163737 , 0.64949544, 0.60335286])

In [995]:
clfRFC.fit(X_train, y_train)

pred_prob = clfRFC.predict_proba(X_test)

auc_score2 = roc_auc_score(y_test, pred_prob[:,1])

auc_score2

0.6159307566511109

In [996]:
clfSDGC = SGDClassifier()
    
scores = cross_val_score(clfSDGC, X, y, cv=10,scoring='roc_auc')
scores

array([0.62995951, 0.50947368, 0.56809717, 0.60291498, 0.54315789,
       0.60809717, 0.59303644, 0.6171875 , 0.49414062, 0.49544271])

### 7. Final model selection and discussion of your model choice and the model weaknesses (generally, where model doesn’t perform well, etc.) [10 pts]

I used cross_val_score to evaluate the quality of predictions from different algorithms
In general, I observed that most of the models, performed very similar with different algorithms.
However the GradientBoostClassifier performed better, reaching 0.70 AUC in one of the cross validation tests.

The RandomForestClassifier very similar, reaching 0.70 AUC in cross validation.

Tunning the number of estimators parameter definetly improved the performance, but adding to many estimators turned the prediction slow. I also found that there is a trade-off between the number of estimators and the learning rate.

I think that the number of features influenced in the scoring because the text features probably don't influence much to improve the performance and they could also be reducing the accuracy.

# Part B

For Part B, the data in the file must go through the preprocessing functions I created in order to drop columns, replace missing values, standarize numeric columns, clean the text from stop words, transform text to numbers (TF-IDF) and  generate the metamodel for text.

In the original model I decided to use the GradientBoostClassifier model with these parameters:

* GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=8, random_state=0, max_features='sqrt', subsample=0.6).

During corss validation, this model hit 0.7 AUC. This happened during one iteration using GBC.

In the case of Random Forest classifier, it hit 0.71 AUC in cross validation.

However, the nature of this algorithms is randomness, so I wasn't aware that my code was creating different models everytime it was running. That was the main reason that the models changed AUC with every run.

The new model is much better and more stable since randomness is controlled.

* GradientBoostingClassifier(n_estimators=160, learning_rate=0.01,max_depth=40, subsample=0.6, random_state = 1, tol=0.1)

This model reaches 0.73 AUC in cross validation, and AUC 0.63... with all the training data.

I know there is much noise in the data, and I removed more columns for this new test, which actually improved the scores.

In [997]:
df_scoring = pd.read_csv('2k_diabetes_scoring.csv')

In [998]:
df_scoring.columns

Index(['race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide.metformin', 'glipizide.metformin',
       'glimepiride.pioglitazone', 'metformin.rosiglitazone',
       'metformin.pioglitazone', 'change', 'diabetesMed', 'diag_1_desc',
       'diag_2_desc', 'diag_3_desc'],
      dtype='object')

In [999]:
# Preprocess the data data for scoring


df_scoring = dropAndReplaceData(df_scoring)

df_scoring = lematization(df_scoring)

X = df_scoring


# Encode categorical data and standarize numerical data 


X_cat_num = pipeline.transform(X)

# Reusing cls

# predicted 
text_pred_model1 = cls.predict_proba(X['diag_1_desc'])[:,1]
#text_pred_model1 = cls.predict(X_test['Processed_diag_1_desc'])

# predicted 
text_pred_model2 = cls.predict_proba(X['diag_2_desc'])[:,1]
#text_pred_model2 = cls.predict(X_test['Processed_diag_2_desc'])

# predicted 
text_pred_model3 = cls.predict_proba(X['diag_3_desc'])[:,1]
#text_pred_model3 = cls.predict(X_test['Processed_diag_3_desc'])

X = joinData(X_cat_num,text_pred_model1,text_pred_model2,text_pred_model3)

#score = clf.score(X, y)



In [1013]:
#pred = clf.predict_proba(X)[:,1]

# Generate predictions using 
clfGBC = GradientBoostingClassifier(n_estimators=160, learning_rate=0.01,max_depth=40, subsample=0.6, random_state = 1, tol=0.1)

clfGBC.fit(X_train, y_train)

pred = clfGBC.predict(X)

In [1014]:
pred_prob = clfGBC.predict_proba(X_test)

auc_score3 = roc_auc_score(y_test, pred_prob[:,1])

auc_score3

0.6212356906385864

In [1015]:
clfGBC.score(X_test, y_test)

0.63125

#### Export the results

In [1016]:
readmitted=pd.DataFrame(data=pred,columns = ['readmitted'])

df_scoring.join(readmitted).to_csv('Freire_Diego_pred2.csv')

df_scoring.join(readmitted)

race  gender  age admission_type_id  \
0     AfricanAmerican  Female   75     Not Available   
1           Caucasian    Male   85     Not Available   
2           Caucasian    Male   35            Urgent   
3           Caucasian    Male   55         Emergency   
4           Caucasian  Female   75         Emergency   
...               ...     ...  ...               ...   
1995        Caucasian  Female   65          Elective   
1996  AfricanAmerican    Male   65            Urgent   
1997  AfricanAmerican  Female   75         Emergency   
1998        Caucasian    Male   85            Urgent   
1999        Caucasian    Male   75          Elective   

                               discharge_disposition_id  \
0                                    Discharged to home   
1     Discharged/transferred to home with home healt...   
2                                    Discharged to home   
3                                    Discharged to home   
4                                    Discharged to home   
...                                                 ...   
1995  Discharged/transferred to home with home healt...   
1996                      Discharged/transferred to SNF   
1997  Discharged/transferred to home with home healt...   
1998                      Discharged/transferred to SNF   
1999                                 Discharged to home   

           admission_source_id  time_in_hospital  num_lab_procedures  \
0           Physician Referral                 7                   1   
1                   Not Mapped                 8                  21   
2           Physician Referral                 3                  37   
3     Transfer from a hospital                 4                  32   
4               Emergency Room                 7                  37   
...                        ...               ...                 ...   
1995        Physician Referral                 3                  30   
1996            Emergency Room                 8                   1   
1997            Emergency Room                13                  46   
1998            Emergency Room                 2                  62   
1999        Physician Referral                 8                  61   

      num_procedures  num_medications  ...  glyburide  pioglitazone  \
0                  0               10  ...         No            No   
1                  0               15  ...         No        Steady   
2                  0               19  ...         No            No   
3                  2                7  ...         No            No   
4                  0               14  ...     Steady            No   
...              ...              ...  ...        ...           ...   
1995               1               29  ...         No            No   
1996               5               15  ...         No            No   
1997               2               14  ...         No            No   
1998               1                7  ...         No            No   
1999               0               18  ...         No            No   

      rosiglitazone insulin change diabetesMed  \
0                No      Up     Ch         Yes   
1                No      No     Ch         Yes   
2                No      No     Ch         Yes   
3                No      No     No          No   
4                No  Steady     Ch         Yes   
...             ...     ...    ...         ...   
1995             No      No     Ch         Yes   
1996         Steady  Steady     Ch         Yes   
1997             No      No     No          No   
1998             No      No     No          No   
1999             No      No     Ch         Yes   

                                            diag_1_desc  \
0                depressive disorder elsewhere classify   
1                  congestive heart failure unspecified   
2                          extrinsic asthma unspecified   
3     femoral hernia obstruction unilateral unspecif...   
4     cerebral thromb